<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">LangChain for Multimodal Apps: Chat With Text/Image Data</h1>
    </div>
</div>

In [7]:
!pip cache purge

Files removed: 451


In [8]:
!pip install git+https://github.com/openai/CLIP.git --quiet
!pip install langchain-community --quiet
!pip install torch --quiet

In [13]:
import clip
import numpy as np
import pandas as pd
import requests
import torch
import warnings

from io import BytesIO
from IPython.display import Image, display
from langchain_community.vectorstores import SingleStoreDB
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from PIL import Image as PILImage

warnings.filterwarnings("ignore")

In [14]:
# Load CLIP model and preprocess function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device = device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 91.3MiB/s]


In [15]:
# Base GitHub directory
base_url = "https://github.com/VeryFatBoy/clip-demo/raw/main/thumbnails/"

# Image filenames
image_filenames = [
    "1_what_makes_singlestore_unique.png",
    "2_streaming_data_ingestion.png",
    "3_what_makes_singlestore_fast.png"
]

images = []

for filename in image_filenames:
    image_url = f"{base_url}{filename}"

    try:
        response = requests.get(image_url)
        response.raise_for_status()

        display(Image(url = image_url))

        image = preprocess(
            PILImage.open(
                BytesIO(response.content)
            )
        ).unsqueeze(0).to(device)

        images.append(image)
    except requests.exceptions.RequestException as e:
        print(f"Failed to load image {filename}: {e}")

# The images list now contains all the processed images
print(f"Preprocessed {len(images)} images.")

Preprocessed 3 images.


In [16]:
texts = [
    "What makes SingleStoreDB unique",
    "Streaming data ingestion",
    "What makes SingleStoreDB fast?",
    "Ultra-Fast Ingestion",
    "Pipelines"
]

In [17]:
# Encode image and text features
with torch.no_grad():
    image_features = model.encode_image(
        torch.cat(images, dim = 0).to(device)
    )
    text_features = model.encode_text(
        clip.tokenize(texts).to(device)
    )

In [18]:
# Normalise features
image_features /= image_features.norm(dim = -1, keepdim = True)
text_features /= text_features.norm(dim = -1, keepdim = True)

In [19]:
# Combine embeddings
combined_features = torch.cat([
    image_features,
    text_features
], dim = 0).cpu().numpy()

In [20]:
# Create the label column based on image or text
labels = ["image"] * image_features.shape[0] + ["text"] * text_features.shape[0]

values = image_filenames + texts

# Create the DataFrame with embeddings and labels
df = pd.DataFrame({
    "vector": list(combined_features),
    "label": labels,
    "value": values
})

In [21]:
df.head(10)

,vector,label,value
0,"[0.027838603, 0.0075516375, -0.019468015, -0.0...",image,1_what_makes_singlestore_unique.png
1,"[-0.022365604, -0.017130833, 0.0041827937, 0.0...",image,2_streaming_data_ingestion.png
2,"[0.007973578, -0.011108348, 0.006702006, -0.01...",image,3_what_makes_singlestore_fast.png
3,"[0.00708297, 0.016323391, 0.014268127, -0.0328...",text,What makes SingleStoreDB unique
4,"[-0.020437056, -0.00872741, 0.0016395436, -0.0...",text,Streaming data ingestion
5,"[0.022778656, 0.0024594143, 0.0021182986, -0.0...",text,What makes SingleStoreDB fast?
6,"[-0.021840213, -0.013803933, -0.0054606185, 0....",text,Ultra-Fast Ingestion
7,"[-0.0011502203, 0.0028256278, -0.032097112, 0....",text,Pipelines


In [23]:
dimensions = len(df.at[0, "vector"])

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [24]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [25]:
%%sql
%%sql
DROP TABLE IF EXISTS langchain_docs;

++
||
++
++

In [26]:
# Define a simple embedding class that returns the embeddings directly
class PrecomputedEmbeddings:
    def embed_documents(self, documents):
        return embeddings_array

    def embed_query(self, query):
        raise NotImplementedError("PrecomputedEmbeddings does not support query embeddings.")

In [27]:
labels = df["label"].tolist()
values = df["value"].tolist()
embeddings_array = df["vector"].tolist()

docs = [
    Document(
        page_content = f"{label}",
        metadata = {"value": value}
    )
    for label, value in zip(labels, values)
]

embeddings = PrecomputedEmbeddings()

docsearch = SingleStoreDB.from_documents(
     docs,
     embeddings,
     table_name = "langchain_docs",
     distance_strategy = DistanceStrategy.DOT_PRODUCT,
     use_vector_index = True,
     vector_size = dimensions
)

In [28]:
%%sql
%%sql
DESCRIBE langchain_docs;

4 rows affected.

Field,Type,Null,Key,Default,Extra
id,bigint(20),NO,PRI,None,auto_increment
content,longtext,YES,,None,
vector,"vector(512, F32)",NO,MUL,None,
metadata,JSON,YES,,None,


In [30]:
%%sql
%%sql
SHOW INDEX FROM langchain_docs;

3 rows affected.

Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Index_options
langchain_docs,0,PRIMARY,1,id,None,None,None,None,,COLUMNSTORE HASH,,,
langchain_docs,1,vector,1,vector,None,None,None,None,,VECTOR,,,"{""metric_type"": ""DOT_PRODUCT""}"
langchain_docs,1,__SHARDKEY,1,id,None,None,None,None,,METADATA_ONLY,,,


In [31]:
%config SqlMagic.named_parameters = "enabled"

## Text Query

In [32]:
def get_text_query_vector(text_query, model, device):
    """
    Encodes a text query into a vector using the CLIP model.

    Args:
    - text_query (str): The text query to encode.
    - model: The preloaded CLIP model.
    - device: The device to use ('cpu' or 'cuda').

    Returns:
    - np.ndarray: The text query vector as a NumPy array.
    """
    with torch.no_grad():
        text_query_features = model.encode_text(
            clip.tokenize(text_query).to(device)
        )

    text_query_features /= text_query_features.norm(dim = -1, keepdim = True)

    return text_query_features.cpu().numpy().astype(np.float32)

In [33]:
text_query = ["What makes SingleStoreDB unique"]

text_query_vector = get_text_query_vector(text_query, model, device)

In [34]:
%%sql
%%sql
SELECT content,
    ROUND(vector <*> :text_query_vector, 5) AS similarity,
    metadata
FROM langchain_docs
ORDER BY similarity DESC;

8 rows affected.

content,similarity,metadata
text,1.0,{'value': 'What makes SingleStoreDB unique'}
text,0.93398,{'value': 'What makes SingleStoreDB fast?'}
text,0.83764,{'value': 'Streaming data ingestion'}
text,0.81279,{'value': 'Ultra-Fast Ingestion'}
text,0.77268,{'value': 'Pipelines'}
image,0.26589,{'value': '1_what_makes_singlestore_unique.png'}
image,0.24434,{'value': '3_what_makes_singlestore_fast.png'}
image,0.24008,{'value': '2_streaming_data_ingestion.png'}


## Image Query

In [35]:
def get_image_query_vector(response, model, device, preprocess):
    """
    Encodes an image from a response content into a vector using the CLIP model.

    Args:
    - response: The HTTP response object containing the image content.
    - model: The preloaded CLIP model.
    - device: The device to use ('cpu' or 'cuda').
    - preprocess: The preprocessing function for the CLIP model.

    Returns:
    - np.ndarray: The image query vector as a NumPy array.
    """
    image = preprocess(
        PILImage.open(
            BytesIO(response.content)
        )
    ).unsqueeze(0).to(device)

    with torch.no_grad():
        image_query_features = model.encode_image(image)

    image_query_features /= image_query_features.norm(dim = -1, keepdim = True)

    return image_query_features.cpu().numpy().astype(np.float32)

In [36]:
image_url = "https://github.com/VeryFatBoy/clip-demo/raw/main/thumbnails/1_what_makes_singlestore_unique.png"
response = requests.get(image_url)
display(Image(url = image_url))

image_query_vector = get_image_query_vector(response, model, device, preprocess)

In [37]:
%%sql
%%sql
SELECT content,
    ROUND(vector <*> :image_query_vector, 5) AS similarity,
    metadata
FROM langchain_docs
ORDER BY similarity DESC;

8 rows affected.

content,similarity,metadata
image,1.0,{'value': '1_what_makes_singlestore_unique.png'}
image,0.88913,{'value': '3_what_makes_singlestore_fast.png'}
image,0.52453,{'value': '2_streaming_data_ingestion.png'}
text,0.2709,{'value': 'What makes SingleStoreDB fast?'}
text,0.26589,{'value': 'What makes SingleStoreDB unique'}
text,0.23276,{'value': 'Streaming data ingestion'}
text,0.15518,{'value': 'Ultra-Fast Ingestion'}
text,0.15302,{'value': 'Pipelines'}


## Combined Query

In [38]:
combined_query_vector = (text_query_vector + image_query_vector) / 2

In [39]:
%%sql
%%sql
SELECT content,
    ROUND(vector <*> :combined_query_vector, 5) AS similarity,
    metadata
FROM langchain_docs
ORDER BY similarity DESC;

8 rows affected.

content,similarity,metadata
image,0.63294,{'value': '1_what_makes_singlestore_unique.png'}
text,0.63294,{'value': 'What makes SingleStoreDB unique'}
text,0.60244,{'value': 'What makes SingleStoreDB fast?'}
image,0.56673,{'value': '3_what_makes_singlestore_fast.png'}
text,0.5352,{'value': 'Streaming data ingestion'}
text,0.48398,{'value': 'Ultra-Fast Ingestion'}
text,0.46285,{'value': 'Pipelines'}
image,0.38231,{'value': '2_streaming_data_ingestion.png'}


## Miscellaneous

In [40]:
text_query = ["A fast military jet"]

text_query_vector = get_text_query_vector(text_query, model, device)

In [42]:
%%sql
%%sql
SELECT content,
    ROUND(vector <*> :text_query_vector, 5) AS similarity,
    metadata
FROM langchain_docs
WHERE content = 'image'
ORDER BY similarity DESC;

3 rows affected.

content,similarity,metadata
image,0.18249,{'value': '3_what_makes_singlestore_fast.png'}
image,0.11514,{'value': '1_what_makes_singlestore_unique.png'}
image,0.09818,{'value': '2_streaming_data_ingestion.png'}
